run with `popstrucure` env

In [1]:
import os
import vcf
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib import cm
import pandas as pd
from matplotlib.pyplot import figure
import glob
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import statistics
import allel
from collections import defaultdict
from itertools import combinations
from haversine import haversine
from skbio.stats.distance import mantel
from scipy.stats import ttest_ind
import cartopy.crs as ccrs
import cartopy.feature as cf
from pykrige.ok import OrdinaryKriging

Unable to determine R home: [Errno 2] No such file or directory: 'R'


In [2]:
proj_dir="/master/nplatt/sch_hae_scan"
results_dir="{}/results".format(proj_dir)

os.chdir(proj_dir)

In [3]:
info_df=pd.read_csv("{}/data/seq_and_sample_docs/all_sh_sb_sample_data.csv".format(proj_dir), sep=",") 

# Mapping rates

In [4]:
Path("{}/mapping_rates".format(results_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/mapping_rates".format(results_dir))

In [5]:
pca_df=pd.read_csv("../initial_pca/pca_df.csv")

In [8]:
pca_df

,sample_name,1,2,3,4,5,6,7,8,9,...,locale,host,phenotype,project,data_type,lat,lon,kmeans_group,pca_species,kmeans_label
0,Sb_NG_au_1.2,-0.174693,0.025622,0.001506,0.004149,0.035574,0.055396,0.032718,-0.019520,0.007569,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
1,Sb_NG_au_2.10,-0.173848,0.025251,0.001886,0.005161,0.036499,0.060013,0.032135,-0.019021,0.003526,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
2,Sb_NG_au_2.13,-0.174493,0.025294,0.001699,0.001562,0.037332,0.060250,0.030712,-0.022199,0.003399,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
3,Sb_NG_au_2.5,-0.173862,0.025171,0.003061,-0.001638,0.037943,0.059797,0.031391,-0.015780,0.000577,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
4,Sb_NG_au_2.6,-0.173011,0.025618,0.001527,0.005455,0.038614,0.061741,0.032516,-0.022343,0.009809,...,na,cow,bovis,nigeria,genome,7.067448,6.271139,2,s_bovis,sb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,ssp_niger_libore_167,0.023482,-0.078226,-0.011342,-0.059942,0.017520,-0.043266,0.041449,-0.016997,0.021684,...,na,snail,unk,scan_africa,genome,13.390080,2.197110,0,s_haematobium,sh_nw
167,ssp_niger_libore_168,0.022237,-0.077112,-0.019412,-0.059978,0.006374,-0.004766,0.030555,-0.041763,0.080463,...,na,snail,unk,scan_africa,genome,13.390080,2.197110,0,s_haematobium,sh_nw
168,ssp_niger_libore_169,0.023857,-0.073894,-0.020205,-0.050080,0.015119,-0.015182,0.038133,-0.019263,0.036911,...,na,snail,unk,scan_africa,genome,13.404170,2.188020,0,s_haematobium,sh_nw
169,ssp_niger_libore_173,0.026630,-0.083945,-0.003698,-0.069531,-0.009745,-0.085123,0.095988,-0.108937,0.082528,...,na,snail,unk,scan_africa,genome,13.404170,2.188020,0,s_haematobium,sh_nw


In [9]:
map_df = pd.read_csv(f"{results_dir}/mapped_reads/mapping_rates.csv", sep=",", header=None)
map_df.columns=["sample_name", "map_rate"]
map_df["map_rate"] = map_df['map_rate'].str.rstrip("%").astype(float)/100
map_df

,sample_name,map_rate
0,bovis_ethiopia_SRR13579874,0.7875
1,bovis_keyna_SRR13579878,0.7666
2,bovis_nigeria_SRR13579877,0.7650
3,bovis_senegal_SRR13579876,0.7638
4,bovis_spain_SRR13579873,0.7635
...,...,...
206,ssp_niger_libore_167,0.8637
207,ssp_niger_libore_168,0.8970
208,ssp_niger_libore_169,0.8977
209,ssp_niger_libore_173,0.8908


In [10]:
map_df = pd.merge(map_df, pca_df[["sample_name", "kmeans_label"]], on='sample_name', how='outer').dropna().reset_index(drop=True)
map_df

,sample_name,map_rate,kmeans_label
0,bovis_ethiopia_SRR13579874,0.7875,sb
1,bovis_keyna_SRR13579878,0.7666,sb
2,bovis_senegal_SRR13579876,0.7638,sb
3,bovis_tanzania_ERR103048,0.7918,sb
4,bovis_tanzania_SRR7867225,0.7755,sb
...,...,...,...
166,ssp_niger_libore_167,0.8637,sh_nw
167,ssp_niger_libore_168,0.8970,sh_nw
168,ssp_niger_libore_169,0.8977,sh_nw
169,ssp_niger_libore_173,0.8908,sh_nw


In [11]:
map_df.groupby(by=["kmeans_label"]).agg(['mean', 'count', 'std'])

/tmp/ipykernel_288398/3888416185.py:1: FutureWarning: ['sample_name'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  map_df.groupby(by=["kmeans_label"]).agg(['mean', 'count', 'std'])


map_rate                
                    mean count       std
kmeans_label                            
s_guineensis    0.607500     1       NaN
s_intercalatum  0.727600     1       NaN
s_margrebowiei  0.674500     1       NaN
s_matthei       0.704400     1       NaN
sb              0.776565    23  0.080901
sh_nw           0.826742    84  0.093948
sh_se           0.763060    60  0.142906

In [49]:
#define samples
sh = map_df[map_df['kmeans_label'].isin(["sh_nw", "sh_se"])]
sb = map_df[map_df['kmeans_label']=='sb']

#perform independent two sample t-test
ttest_ind(sh['map_rate'], sb['map_rate'])

Ttest_indResult(statistic=0.8499875241109823, pvalue=0.3966021310889227)

In [77]:
idv_df=pd.read_csv("../filter_genotypes/indv_gt_freq.tbl", sep="\t")
idv_df

,INDV,N_DATA,N_GENOTYPES_FILTERED,N_MISS,F_MISS
0,Sb_NG_ak_2.1,38198427,0,3256551,0.085253
1,Sb_NG_au_1.2,38198427,0,4007156,0.104904
2,Sb_NG_au_2.10,38198427,0,3597035,0.094167
3,Sb_NG_au_2.13,38198427,0,3738947,0.097882
4,Sb_NG_au_2.5,38198427,0,3616554,0.094678
...,...,...,...,...,...
214,ssp_niger_libore_167,38198427,0,8932549,0.233846
215,ssp_niger_libore_168,38198427,0,3975609,0.104078
216,ssp_niger_libore_169,38198427,0,597245,0.015635
217,ssp_niger_libore_173,38198427,0,1491990,0.039059


In [78]:
idv_df = pd.merge(idv_df, pca_df[["sample_name", "kmeans_label"]], left_on='INDV', right_on='sample_name', how='outer').dropna().reset_index(drop=True)
idv_df

,INDV,N_DATA,N_GENOTYPES_FILTERED,N_MISS,F_MISS,sample_name,kmeans_label
0,Sb_NG_au_1.2,38198427,0,4007156,0.104904,Sb_NG_au_1.2,sb
1,Sb_NG_au_2.10,38198427,0,3597035,0.094167,Sb_NG_au_2.10,sb
2,Sb_NG_au_2.13,38198427,0,3738947,0.097882,Sb_NG_au_2.13,sb
3,Sb_NG_au_2.5,38198427,0,3616554,0.094678,Sb_NG_au_2.5,sb
4,Sb_NG_au_2.6,38198427,0,3299502,0.086378,Sb_NG_au_2.6,sb
...,...,...,...,...,...,...,...
166,ssp_niger_libore_167,38198427,0,8932549,0.233846,ssp_niger_libore_167,sh_nw
167,ssp_niger_libore_168,38198427,0,3975609,0.104078,ssp_niger_libore_168,sh_nw
168,ssp_niger_libore_169,38198427,0,597245,0.015635,ssp_niger_libore_169,sh_nw
169,ssp_niger_libore_173,38198427,0,1491990,0.039059,ssp_niger_libore_173,sh_nw


In [79]:
idv_df.groupby(by=["kmeans_label"])["F_MISS"].agg(['mean', 'count', 'std'])

,mean,count,std
kmeans_label,,,
s_guineensis,0.376764,1,NaN
s_intercalatum,0.083733,1,NaN
s_margrebowiei,0.231996,1,NaN
s_matthei,0.100289,1,NaN
sb,0.161157,23,0.105172
sh_nw,0.154415,84,0.129582
sh_se,0.140557,60,0.125487


In [80]:
#define samples
sh = idv_df[idv_df['kmeans_label'].isin(["sh_nw", "sh_se"])]
sb = idv_df[idv_df['kmeans_label']=='sb']

#perform independent two sample t-test
ttest_ind(sh['F_MISS'], sb['F_MISS'])

Ttest_indResult(statistic=-0.4463605776958125, pvalue=0.6559220423447343)